In [1]:
from ontology_v6 import graph, getOnto
from owlQuery import fill_template_with_relationships

from owlready2 import *
import random

from util import save_as_csv, replace_relations, identify_logical_operators

onto = getOnto()

ok-onto6
ok-onto6


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())

        question_results = []
        for random_food in all_foods:
            for related_recipe in random_food.is_ingredient_of:
                for related_taste in random_food.has_taste:
                    template = random.choice(templates)

                    question_result = template.replace('{food}', f"{random_food.name}")
                    question_result = question_result.replace('{taste}', related_taste.name)
                    question_result = question_result.replace('{recipe}', related_recipe.name)

                    same_class_same_taste_foods = [
                        food.name 
                        for food in onto.Food.instances() 
                        if (
                            food is not random_food 
                            and related_taste in food.has_taste
                        )
                    ]

                    possible_answers = same_class_same_taste_foods
                    question_results.append((question_result, possible_answers))

    return question_results

templates = ["What can be used instead of {food} for {recipe} if the goal is to achieve a {taste} flavor?",
            "If {recipe} calls for a {taste} flavour but {food} cannot be used, what alternative is available?"]
results = generate_all_question_results(templates)

save_as_csv(results, "question_results_2.csv")


Results saved to question_results_2.csv


In [22]:
# not used
def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())

        question_results = []
        for random_food in all_foods:
            for related_recipe in random_food.is_ingredient_of:
                for related_taste in random_food.has_taste:
                    template = random.choice(templates)

                    question_result = template.replace('{food}', f"{random_food.name}")
                    question_result = question_result.replace('{taste}', related_taste.name)
                    question_result = question_result.replace('{recipe}', related_recipe.name)

                    same_class_same_taste_foods = [
                        food.name 
                        for food in onto.Food.instances() 
                        if (
                            food is not random_food 
                            and related_taste in food.has_taste
                        )
                    ]

                    possible_answers = same_class_same_taste_foods if same_class_same_taste_foods else ['None']
                    question_results.append((question_result, possible_answers))

    return question_results

templates = ["Suppose I want a {taste} tasting {class} that is also {colour} coloured, what can I eat other than {food}?",
            "If {food} is not suitable for me, what other {class} with {colour} colour and {taste} taste can I choose?",
            "Looking for alternatives to {food}, recommend a {colour} {class} that has a {taste} flavor."]
results = generate_all_question_results(templates)

save_as_csv(results, "question_results_3-old.csv")


Results saved to question_results_3.csv


In [3]:

def generate_all_question_results(templates):
    with onto:
        all_classes = list(onto.classes())
        all_foods = list(onto.Food.instances())

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_food in ont_class.instances():
                    for related_taste in random_food.has_taste:
                        for related_color in random_food.has_color:
                            template = random.choice(templates)

                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{food}', random_food.name)
                            question_result = question_result.replace('{taste}', related_taste.name)
                            question_result = question_result.replace('{colour}', related_color.name)
                            
                            has_taste = [food.name for food in all_foods if related_taste in food.has_taste]
                            # is_class = [food.name for food in all_foods if type(food) == type(random_food)]
                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            is_colour = [food.name for food in all_foods if related_color in food.has_color]
                            fa = [food for food in has_taste if food in is_class and food in is_colour and food is not random_food.name]

                            question_results.append((question_result, has_taste, is_class, is_colour, fa))

    return question_results

templates = ["Suppose I want a {taste} tasting {class} that is also {colour} coloured, what can I eat other than {food}?",
            "If {food} is not suitable for me, what other {class} with {colour} colour and {taste} taste can I choose?",
            "Looking for alternatives to {food}, recommend a {colour} {class} that has a {taste} flavor."]
results = generate_all_question_results(templates)

save_as_csv(results, "question_results_3.csv", ["Question", "has_taste", "is_class", "is_colour", "fa"])


Results saved to question_results_3.csv


In [4]:
#"Considering the unavailability of {food}, which {class} with a similar {nutrition} content would be ideal for making {recipe}?"
def generate_all_question_results(templates):
    with onto:
        all_classes = list(onto.classes())
        all_foods = list(onto.Food.instances())

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_food in ont_class.instances():
                    for related_nutrition in random_food.has_nutrition:
                        for related_recipe in random_food.is_ingredient_of:
                            template = random.choice(templates)

                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{food}', random_food.name)
                            question_result = question_result.replace('{nutrition}', related_nutrition.name)
                            question_result = question_result.replace('{recipe}', related_recipe.name)

                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_nutrition = [food.name for food in all_foods if related_nutrition in food.has_nutrition]
                            fa = [food for food in is_class if food in has_nutrition and food is not random_food.name]

                            question_results.append((question_result, is_class, has_nutrition, fa))

    return question_results

template = "Considering the unavailability of {food}, which {class} with a similar {nutrition} content would be ideal for making {recipe}?"
templates = [
    "In the context of {recipe} preparation, if {food} is unavailable, what other {class} with comparable {nutrition} content would be suitable?",
    "For making {recipe}, what alternative {class} with similar {nutrition} content is recommended if {food} is not accessible?",
    "Propose another {class} with a similar {nutrition} profile if {food} is unavailable, suitable for preparing {recipe}.",
    "Considering the unavailability of {food}, which {class} with a similar {nutrition} content would be ideal for making {recipe}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_4.csv", ["Question", "is_class", "has_nutrition", "fa"])


Results saved to question_results_4.csv


In [5]:
# "If {food} is not available, what other food with a similar taste to {taste} can be used for {cooking_method}?"
def generate_all_question_results(template):
    with onto:
        all_classes = list(onto.classes())
        all_foods = list(onto.Food.instances())
        distinct_is_growing_values = set(g for food in onto.Food.instances() for g in getattr(food, 'is_growing', []))


        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_food in ont_class.instances():
                    for related_taste in random_food.has_taste:
                        for is_growing in distinct_is_growing_values:
                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{food}', random_food.name)
                            question_result = question_result.replace('{taste}', related_taste.name)
                            question_result = question_result.replace('{is_growing}', is_growing.name)

                            # same_class_same_taste_foods = [
                            #     food.name 
                            #     for food in onto.Food.instances() 
                            #     if (
                            #         food is not random_food 
                            #         and related_taste in food.has_taste
                            #         and is_growing in food.is_growing
                            #         and isinstance(food, ont_class)
                            #     )
                            # ]

                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_taste = [food.name for food in all_foods if related_taste in food.has_taste]
                            has_growing_style = [food.name for food in all_foods if is_growing in food.is_growing]
                            fa = [food for food in is_class if food in has_taste and food in has_growing_style and food is not random_food.name]

                            # possible_answers = same_class_same_taste_foods if same_class_same_taste_foods else ['None']
                            question_results.append((question_result, is_class, has_taste, has_growing_style, fa))

    return question_results

template = "If {food} is not available, what other {class} with a similar {taste} taste can be used that grows {is_growing}?"
results = generate_all_question_results(template)

save_as_csv(results, "question_results_5.csv", ["Question", "is_class", "has_taste", "has_growing_style,", "fa"])


Results saved to question_results_5.csv


In [6]:
def generate_all_question_results(template):
    with onto:
        all_foods = list(onto.Food.instances())
        all_recipes = list(onto.Recipe.instances())
        all_classes = list(onto.classes())

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_food in ont_class.instances():
                    for related_nutrient in random_food.has_nutrition:
                        if random_food.has_shape:
                            related_shape = random_food.has_shape[0]

                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{food}', f"{random_food.name}")
                            question_result = question_result.replace('{shape}', related_shape.name)
                            question_result = question_result.replace('{nutrient}', related_nutrient.name)

                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_shape = [food.name for food in all_foods if related_shape in food.has_shape]
                            has_nutrient = [food.name for food in all_foods if related_nutrient in food.has_nutrition]
                            fa = [food for food in is_class if food in has_shape and food in has_nutrient and food is not random_food.name]

                            question_results.append((question_result, is_class, has_shape, has_nutrient, fa))
                                
    return question_results

template = "If I don't like {food}, what other {class} is available that also has a {shape} shape and can provide {nutrient}?"
results = generate_all_question_results(template)

save_as_csv(results, "question_results_6.csv", ["Question", "is_class", "has_shape", "has_nutrient", "fa"])

Results saved to question_results_6.csv


In [7]:
# 7. at least x
from owlready2 import *
import random

def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())
        all_nutritions = list(onto.Nutrition.instances())
        all_tastes = list(onto.Taste.instances())
        all_classes = list(onto.classes())

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_taste in all_tastes:
                    for random_nutrient in all_nutritions:
                        template = random.choice(templates)

                        question_result = template.replace('{class}', ont_class.name)
                        question_result = question_result.replace('{taste}', random_taste.name)
                        question_result = question_result.replace('{nutrient}', random_nutrient.name)

                        not_nutrient = [food.name for food in all_foods if random_nutrient not in food.has_nutrition]
                        is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                        has_number_of_taste = [food.name for food in all_foods if len(food.has_taste) >= 3]
                        not_taste = [food.name for food in all_foods if random_taste not in food.has_taste]
                        fa = [food for food in is_class if food in not_nutrient and food in has_number_of_taste and food in not_taste]

                        question_results.append((question_result, not_nutrient, is_class, has_number_of_taste, not_taste, fa))
                                
    return question_results

# template = "If I am allergic to {nutrient}, which items of {class} are recommended that have at least three tastes but are not {taste}?"
templates = [
    "If I am allergic to {nutrient}, which items of {class} are recommended that have at least three tastes but are not {taste}?",
    "Which {class} options would you suggest for someone allergic to {nutrient}, requiring at least three tastes, if they dislike {taste}?",
    "Considering an aversion to {taste} food, which items from the {class} category would you recommend, requiring a minimum of three tastes and do not contain {nutrient}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_7.csv", [
    "Question",
    "not_nutrient",
    "is_class",
    "has_number_of_taste",
    "not_taste",
    "fa"
])


Results saved to question_results_7.csv


In [8]:
# 8. Multi
from owlready2 import *
import random

# negation = ["Taste", "Colour", "Shape"]

def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())
        all_nutritions = list(onto.Nutrition.instances())
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_classes = list(onto.classes())
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        negation = all_tastes + all_colours + all_shapes

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                # for random_taste in all_tastes:
                for random_nutrient in all_nutritions:
                    for tuple_pair in negation:
                        relation, attributes = tuple_pair
                        for attribute in attributes:
                            template = random.choice(templates)
                            # not_class = random.choice(negation)

                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{attr}', attribute.name)
                            question_result = question_result.replace('{nutrient}', random_nutrient.name)

                            not_nutrient = [food.name for food in all_foods if random_nutrient not in food.has_nutrition]
                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_number_of_taste = [food.name for food in all_foods if len(food.has_taste) >= 3]
                            # not_taste = [food.name for food in all_foods if random_taste not in food.has_taste]
                            # not_attr = [food.name for food in all_foods if attribute in food.[relation]]
                            not_attr = [food.name for food in all_foods if attribute not in getattr(food, relation, [])]
                            fa = [food for food in is_class if food in not_nutrient and food in has_number_of_taste and food in not_attr]

                            question_results.append((question_result, not_nutrient, is_class, has_number_of_taste, not_attr, fa))
                                
    return question_results

templates = [
    "If I am allergic to {nutrient}, which items of {class} are recommended that have at least three tastes but are not {attr}?",
    "Which {class} options would you suggest for someone allergic to {nutrient}, requiring at least three tastes, if they dislike {attr} foods?",
    "Considering an aversion to {attr} food, which items from the {class} category would you recommend, requiring a minimum of three tastes and do not contain {nutrient}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_8.csv", [
    "Question",
    "not_nutrient",
    "is_class",
    "has_number_of_taste",
    "not_attribute",
    "fa"
])


Results saved to question_results_8.csv


In [15]:
# 9. Multi 
from owlready2 import *
import random

# negation = ["Taste", "Colour", "Shape"]

def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())
        all_nutritions = list(onto.Nutrition.instances())
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_classes = list(onto.classes())
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        negation = all_tastes + all_colours + all_shapes

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_nutrient in all_nutritions:
                    for tuple_pair in negation:
                        relation, attributes = tuple_pair
                        for attribute in attributes:
                            template = random.choice(templates)
                            
                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{attr}', attribute.name)
                            question_result = question_result.replace('{nutrient}', random_nutrient.name)

                            not_nutrient = [food.name for food in all_foods if random_nutrient not in food.has_nutrition]
                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_number_of_nutrient = [food.name for food in all_foods if len(food.has_nutrition) >= 3]
                            not_attr = [food.name for food in all_foods if attribute not in getattr(food, relation, [])]
                            fa = [food for food in is_class if food in not_nutrient and food in has_number_of_nutrient and food in not_attr]

                            question_results.append((question_result, not_nutrient, is_class, has_number_of_nutrient, not_attr, fa))
                                
    return question_results

templates = [
    "If I am allergic to {nutrient}, which items of {class} are recommended that have at least three nutrients but are not {attr}?",
    "Which {class} options would you suggest for someone allergic to {nutrient}, requiring at least three nutrients, if they dislike {attr} foods?",
    "Considering an aversion to {attr} food, which items from the {class} category would you recommend, requiring a minimum of three nutrients and do not contain {nutrient}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_9.csv", [
    "Question",
    "not_nutrient",
    "is_class",
    "has_number_of_nutrient",
    "not_attribute",
    "fa"
])


Results saved to question_results_9.csv


In [13]:
# 10. Multi 
from owlready2 import *
import random

# negation = ["Taste", "Colour", "Shape"]

def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())
        all_nutritions = list(onto.Nutrition.instances())
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_classes = list(onto.classes())
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        distinct_edible_part_values = set(g for food in onto.Food.instances() for g in getattr(food, 'has_edible_part', []))

        negation = all_tastes + all_colours + all_shapes

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_edible in distinct_edible_part_values:
                    for tuple_pair in negation:
                        relation, attributes = tuple_pair
                        for attribute in attributes:
                            template = random.choice(templates)
                            
                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{attr}', attribute.name)
                            question_result = question_result.replace('{edible}', random_edible.name)

                            has_edible = [food.name for food in all_foods if random_edible in food.has_edible_part]
                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            has_number_of_nutrient = [food.name for food in all_foods if len(food.has_nutrition) >= 3]
                            not_attr = [food.name for food in all_foods if attribute not in getattr(food, relation, [])]
                            fa = [food for food in is_class if food in has_edible and food in has_number_of_nutrient and food in not_attr]

                            question_results.append((question_result, has_edible, is_class, has_number_of_nutrient, not_attr, fa))
                                
    return question_results

templates = [
    "If I want to avoid {attr} foods, which items of {class} are recommended that have at least three nutrients and has edible {edible}?",
    "Which {class} options would you suggest with edible {edible}, requiring at least three nutrients, and are not {attr}?",
    "Considering an aversion to {attr} food, which items from the {class} category would you recommend, requiring a minimum of three nutrients and having edible {edible}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_10.csv", [
    "Question",
    "has_edible",
    "is_class",
    "has_number_of_nutrient",
    "not_attribute",
    "fa"
])


Results saved to question_results_10.csv


In [12]:
# 10. Multi with OR

def generate_all_question_results(templates):
    with onto:
        all_foods = list(onto.Food.instances())
        all_nutritions = list(onto.Nutrition.instances())
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_classes = list(onto.classes())
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        distinct_edible_part_values = set(g for food in onto.Food.instances() for g in getattr(food, 'has_edible_part', []))

        negation = all_tastes + all_colours + all_shapes

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_edible in distinct_edible_part_values:
                    for tuple_pair in negation:
                        relation, attributes = tuple_pair
                        for attribute in attributes:
                            template = random.choice(templates)
                            
                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{attr}', attribute.name)
                            question_result = question_result.replace('{edible}', random_edible.name)

                            has_edible = [food.name for food in all_foods if random_edible in food.has_edible_part]
                            has_number_of_nutrient = [food.name for food in all_foods if len(food.has_nutrition) >= 3]
                            satisfies_or = list(set(has_edible) | set(has_number_of_nutrient))
                            is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            not_attr = [food.name for food in all_foods if attribute not in getattr(food, relation, [])]
                            fa = [food for food in is_class if food in not_attr and food in satisfies_or]

                            question_results.append((question_result, has_edible, is_class, has_number_of_nutrient, not_attr, fa))
                                
    return question_results

templates = [
    "If I want to avoid {attr} foods, which items of {class} are recommended that have at least three nutrients or have edible {edible}?",
    "Which {class} options would you suggest with either edible {edible} or having at least three nutrients, and also are not {attr}?",
    "Considering an aversion to {attr} food, which items from the {class} category would you recommend, requiring a minimum of three nutrients or having edible {edible}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_10-OR.csv", [
    "Question",
    "has_edible",
    "is_class",
    "has_number_of_nutrient",
    "not_attribute",
    "fa"
])


Results saved to question_results_10-OR.csv


In [9]:
# 11. Multi Fruit operators 
from util import replace_relations, identify_logical_operators

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            for rel_3, attributes_3 in all_relations:
                                if rel_3 != rel_1 and rel_3 != rel_2:
                                    for attribute3 in attributes_3:

                                        template = random.choice(templates)
                                        op1 = random.choice(ops)
                                        op2 = random.choice(ops)
                                        
                                        question_result = template.replace('{rel_a}', rel_1)
                                        question_result = question_result.replace('{attr_a}', attribute1.name)
                                        question_result = question_result.replace('{rel_b}', rel_2)
                                        question_result = question_result.replace('{attr_b}', attribute2.name)
                                        question_result = question_result.replace('{rel_c}', rel_3)
                                        question_result = question_result.replace('{attr_c}', attribute3.name)
                                        question_result = question_result.replace('{op_a}', op1)
                                        question_result = question_result.replace('{op_b}', op2)

                                        def getMatches(rel, attr, op):
                                            if "not" in op:
                                                return [food.name for food in all_fruits if attr not in getattr(food, rel, [])]
                                            else:
                                                return [food.name for food in all_fruits if attr in getattr(food, rel, [])]

                                        def reduceAns(set1, set2, op):
                                            if "and" in op:
                                                return [item for item in set1 if item in set2]
                                            else:
                                                return list(set(set1) | set(set2))

                                        match1 = getMatches(rel_1, attribute1, "")
                                        match2 = getMatches(rel_2, attribute2, op1)
                                        match3 = getMatches(rel_3, attribute3, op2)

                                        step1 = reduceAns(match1, match2, op1)
                                        fa = reduceAns(step1, match3, op2)

                                        qnl = replace_relations(question_result)
                                        operators, level, types = identify_logical_operators(question_result)

                                        question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                    
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Which fruit {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} {rel_c} {attr_c}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_fruit_1.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_dairy_1.csv


In [2]:
# 12. Multi Fruit operators, "at least nutritions"

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]
        ops_not = ["", "not "]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            
                            template = random.choice(templates)
                            not_or_blank = random.choice(ops_not)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            
                            question_result = template.replace('{maybe_not}', not_or_blank)
                            question_result = question_result.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_fruits if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_fruits if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_fruits if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_fruits if len(getattr(food, rel, [])) >= 3]


                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, not_or_blank)
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_nutrition", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            # fa = reduceAns(step1, has_num_nutrition, op2) !!BUG fixed
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Which fruit {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three nutritions?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_fruit_2_revised.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_fruit_2_revised.csv


In [3]:
# 13. Multi Fruit operators, "at least"

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]
        ops_not = ["", "not "]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:

                            template = random.choice(templates)
                            not_or_blank = random.choice(ops_not)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            
                            question_result = template.replace('{maybe_not}', not_or_blank)
                            question_result = question_result.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_fruits if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_fruits if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_fruits if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_fruits if len(getattr(food, rel, [])) >= 3]

                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, not_or_blank)
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_taste", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            # fa = reduceAns(step1, has_num_taste, op2) !!BUG fixed
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types ))
                
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Which fruit {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three tastes?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_fruit_3_revised.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_fruit_3_revised.csv


In [4]:
# 14. Multi Vegetable operators, "at least nutritions" - ALTER

def generate_all_question_results(templates):
    with onto:
        all_vegatables = list(onto.Vegetable.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]

        distinct_edible_part = [("has_edible_part", list(set(g for food in onto.Food.instances() for g in getattr(food, 'has_edible_part', []))))]
        distinct_category = [("is_category_of",list(set(g for food in onto.Food.instances() for g in getattr(food, 'is_category_of', []))))]
        distinct_has_seed = [("has_seed_inside", list(set(g for food in onto.Food.instances() for g in getattr(food, 'has_seed_inside', []))))]

        all_relations = distinct_edible_part + distinct_category + distinct_has_seed # + all_nutritions
        ops = ["and", "or", "and not", "or not"]
        ops_not = ["", "not "]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            
                            template = random.choice(templates)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            not_or_blank = random.choice(ops_not)
                            
                            question_result = template.replace('{maybe_not}', not_or_blank)
                            question_result = question_result.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_vegatables if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_vegatables if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_vegatables if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_vegatables if len(getattr(food, rel, [])) >= 3]


                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, not_or_blank)
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_nutrition", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            # fa = reduceAns(step1, has_num_nutrition, op2) !!BUG fixed
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Which vegetable {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three nutritions?",
    "Identify the vegetable that {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three nutritions?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_vegetable_2_revised.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_vegetable_2_revised.csv


In [5]:
# 15. Multi Vegetable operators, "at least Tastes" - ALTER

def generate_all_question_results(templates):
    with onto:
        all_vegatables = list(onto.Vegetable.instances())

        distinct_edible_part = [("has_edible_part", list(set(g for food in onto.Food.instances() for g in getattr(food, 'has_edible_part', []))))]
        distinct_category = [("is_category_of",list(set(g for food in onto.Food.instances() for g in getattr(food, 'is_category_of', []))))]
        distinct_has_seed = [("has_seed_inside", list(set(g for food in onto.Food.instances() for g in getattr(food, 'has_seed_inside', []))))]

        all_relations = distinct_edible_part + distinct_category + distinct_has_seed
        ops = ["and", "or", "and not", "or not"]
        ops_not = ["", "not "]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            
                            template = random.choice(templates)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            not_or_blank = random.choice(ops_not)
                            
                            question_result = template.replace('{maybe_not}', not_or_blank)
                            question_result = question_result.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_vegatables if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_vegatables if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_vegatables if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_vegatables if len(getattr(food, rel, [])) >= 3]


                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, not_or_blank)
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_taste", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Which vegetable {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three tastes?",
    "Identify the vegetable that {maybe_not}{rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three tastes?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_vegetable_3_revised.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_vegetable_3_revised.csv


In [4]:
# 16. Multi Dairy operators 
from util import replace_relations, identify_logical_operators

def generate_all_question_results(templates):
    with onto:
        all_dairy = list(onto.Dairy.instances())
        # all_nutritions = [("has_nutrition", set(g for food in onto.Dairy.instances() for g in getattr(food, 'has_nutrition', [])))]
        # all_tastes = [("has_taste", set(g for food in onto.Dairy.instances() for g in getattr(food, 'has_taste', [])))]
        # all_colours = [("has_color", set(g for food in onto.Dairy.instances() for g in getattr(food, 'has_color', [])))]
        # all_shapes = [("has_shape", list(onto.Shape.instances()))]
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]

        all_relations = all_tastes + all_colours + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            for rel_3, attributes_3 in all_relations:
                                if rel_3 != rel_1 and rel_3 != rel_2:
                                    for attribute3 in attributes_3:

                                        template = random.choice(templates)
                                        op1 = random.choice(ops)
                                        op2 = random.choice(ops)
                                        
                                        question_result = template.replace('{rel_a}', rel_1)
                                        question_result = question_result.replace('{attr_a}', attribute1.name)
                                        question_result = question_result.replace('{rel_b}', rel_2)
                                        question_result = question_result.replace('{attr_b}', attribute2.name)
                                        question_result = question_result.replace('{rel_c}', rel_3)
                                        question_result = question_result.replace('{attr_c}', attribute3.name)
                                        question_result = question_result.replace('{op_a}', op1)
                                        question_result = question_result.replace('{op_b}', op2)

                                        def getMatches(rel, attr, op):
                                            if "not" in op:
                                                return [food.name for food in all_dairy if attr not in getattr(food, rel, [])]
                                            else:
                                                return [food.name for food in all_dairy if attr in getattr(food, rel, [])]

                                        def reduceAns(set1, set2, op):
                                            if "and" in op:
                                                return [item for item in set1 if item in set2]
                                            else:
                                                return list(set(set1) | set(set2))

                                        match1 = getMatches(rel_1, attribute1, "")
                                        match2 = getMatches(rel_2, attribute2, op1)
                                        match3 = getMatches(rel_3, attribute3, op2)

                                        step1 = reduceAns(match1, match2, op1)
                                        fa = reduceAns(step1, match3, op2)

                                        qnl = replace_relations(question_result)
                                        operators, level, types = identify_logical_operators(question_result)

                                        question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                    
    return question_results

templates = [
    # "Which dairy has taste {taste} and has shape {shape} or has colour {colour}",
    "Which dairy {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} {rel_c} {attr_c}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_dairy_1.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_dairy_1.csv


In [2]:
# 17. Multi Dairy operators, "at least nutritions"

def generate_all_question_results(templates):
    with onto:
        all_dairy = list(onto.Dairy.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]

        all_relations = all_tastes + all_colours + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            
                            template = random.choice(templates)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            
                            question_result = template.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_dairy if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_dairy if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_dairy if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_dairy if len(getattr(food, rel, [])) >= 3]


                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, "")
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_nutrition", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            # fa = reduceAns(step1, has_num_nutrition, op2) !!BUG fixed
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                
    return question_results

templates = [
    # "Which dairy has taste {taste} and has shape {shape} or has colour {colour}",
    "Which dairy {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three nutritions?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_dairy_2.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_dairy_2.csv


In [2]:
# 18. Multi Dairy operators, "at least nutritions"

def generate_all_question_results(templates):
    with onto:
        all_dairy = list(onto.Dairy.instances()) + list(onto.Poultry.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]

        all_relations = all_tastes + all_colours + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            
                            template = random.choice(templates)
                            op1 = random.choice(ops)
                            op2 = random.choice(ops)
                            
                            question_result = template.replace('{rel_a}', rel_1)
                            question_result = question_result.replace('{attr_a}', attribute1.name)
                            question_result = question_result.replace('{rel_b}', rel_2)
                            question_result = question_result.replace('{attr_b}', attribute2.name)
                            question_result = question_result.replace('{op_a}', op1)
                            question_result = question_result.replace('{op_b}', op2)

                            def getMatches(rel, attr, op):
                                if "not" in op:
                                    return [food.name for food in all_dairy if attr not in getattr(food, rel, [])]
                                else:
                                    return [food.name for food in all_dairy if attr in getattr(food, rel, [])]

                            def getMatchesNum(rel, op):
                                if "not" in op:
                                    return [food.name for food in all_dairy if not len(getattr(food, rel, [])) >= 3]
                                else:
                                    return [food.name for food in all_dairy if len(getattr(food, rel, [])) >= 3]


                            def reduceAns(set1, set2, op):
                                if "and" in op:
                                    return [item for item in set1 if item in set2]
                                else:
                                    return list(set(set1) | set(set2))

                            match1 = getMatches(rel_1, attribute1, "not")
                            match2 = getMatches(rel_2, attribute2, op1)
                            match3 = getMatchesNum("has_nutrition", op2)
                            
                            step1 = reduceAns(match1, match2, op1)
                            fa = reduceAns(step1, match3, op2)

                            qnl = replace_relations(question_result)
                            operators, level, types = identify_logical_operators(question_result)

                            question_results.append((question_result, qnl, match1, match2, match3, fa, operators, level, types))
                
    return question_results

templates = [
    # "Which dairy has taste {taste} and has shape {shape} or has colour {colour}",
    "Which dairy or poultry product not {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} has at least three nutritions?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_dairy_3.csv", [
    "Question",
    "questions_nlp",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])

Results saved to question_results_dairy_3.csv


In [3]:
# Check the count of non-empty 'fa' lists
non_empty_fa_count = sum(1 for result in results if result[-4])  # Assuming 'fa' is the last column
print("fa count:", sum(1 for result in results))
print("Non-empty fa count:", non_empty_fa_count)


fa count: 1758
Non-empty fa count: 1138
